# ICP via ITK Heart Model to Image Registration Experiment

This notebook demonstrates using the `RegisterModelToImageICPITK` class to register
a statistical shape model to patient CT images using PCA-based shape variation.

## Overview
- Uses the KCL Heart Model PCA statistical shape model
- Registers to the same Duke Heart CT data as the original notebook
- Converts segmentation mask to intensity image for registration

## Setup and Imports

In [1]:
# PCA-based Heart Model to Image Registration Experiment

import os
from pathlib import Path

import itk
import numpy as np
import pyvista as pv
from itk import TubeTK as ttk

# Import from PhysioMotion4D package
from physiomotion4d import (
    ContourTools,
    RegisterModelsICPITK,
    TransformTools,
)

## Define File Paths

In [2]:
# Patient CT image (defines coordinate frame)
patient_data_dir = Path.cwd().parent.parent / "data" / "Slicer-Heart-CT"
patient_ct_path = patient_data_dir / "patient_img.mha"
patient_ct_heart_mask_path = patient_data_dir / "patient_heart_wall_mask.nii.gz"

# heart model data
heart_model_data_dir = Path.cwd().parent.parent / "data" / "KCL-Heart-Model"
heart_model_path = heart_model_data_dir / "average_mesh.vtk"

# Output directory
output_dir = Path.cwd() / "results_icp_itk"
os.makedirs(output_dir, exist_ok=True)

print(f"Patient data: {patient_data_dir}")
print(f"Output directory: {output_dir}")

Patient data: C:\src\Projects\PhysioMotion\physiomotion4d\data\Slicer-Heart-CT
Output directory: C:\src\Projects\PhysioMotion\physiomotion4d\experiments\Heart-Model_To_Patient\results_icp_itk


## Load and Preprocess Patient Image

In [3]:
# Load patient CT image
print("Loading patient CT image...")
patient_image = itk.imread(str(patient_ct_path))
print(f"  Original size: {itk.size(patient_image)}")
print(f"  Original spacing: {itk.spacing(patient_image)}")

# Resample to 1mm isotropic spacing
print("Resampling to sotropic...")
resampler = ttk.ResampleImage.New(Input=patient_image)
resampler.SetMakeHighResIso(True)
resampler.Update()
patient_image = resampler.GetOutput()

print(f"  Resampled size: {itk.size(patient_image)}")
print(f"  Resampled spacing: {itk.spacing(patient_image)}")

# Save preprocessed image
itk.imwrite(patient_image, str(output_dir / "patient_image.mha"), compression=True)
print("✓ Saved preprocessed image")

Loading patient CT image...
  Original size: itkSize3 ([307, 234, 152])
  Original spacing: itkVectorD3 ([1, 1, 1])
Resampling to sotropic...


  Resampled size: itkSize3 ([307, 234, 152])
  Resampled spacing: itkVectorD3 ([1, 1, 1])
✓ Saved preprocessed image


## Load and Process Heart Segmentation Mask

In [4]:
# Load heart segmentation mask
print("Loading heart segmentation mask...")
patient_heart_mask = itk.imread(str(patient_ct_heart_mask_path))

print(f"  Mask size: {itk.size(patient_heart_mask)}")
print(f"  Mask spacing: {itk.spacing(patient_heart_mask)}")

Loading heart segmentation mask...
  Mask size: itkSize3 ([307, 234, 152])
  Mask spacing: itkVectorD3 ([1, 1, 1])


In [5]:
# Handle image orientation (flip if needed)
flip0 = np.array(patient_image.GetDirection())[0, 0] < 0
flip1 = np.array(patient_image.GetDirection())[1, 1] < 0
flip2 = np.array(patient_image.GetDirection())[2, 2] < 0

if flip0 or flip1 or flip2:
    print(f"Flipping image axes: {flip0}, {flip1}, {flip2}")

    # Flip CT image
    flip_filter = itk.FlipImageFilter.New(Input=patient_image)
    flip_filter.SetFlipAxes([int(flip0), int(flip1), int(flip2)])
    flip_filter.SetFlipAboutOrigin(True)
    flip_filter.Update()
    patient_image = flip_filter.GetOutput()
    id_mat = itk.Matrix[itk.D, 3, 3]()
    id_mat.SetIdentity()
    patient_image.SetDirection(id_mat)

    # Flip mask image
    flip_filter = itk.FlipImageFilter.New(Input=patient_heart_mask)
    flip_filter.SetFlipAxes([int(flip0), int(flip1), int(flip2)])
    flip_filter.SetFlipAboutOrigin(True)
    flip_filter.Update()
    patient_heart_mask = flip_filter.GetOutput()
    patient_heart_mask.SetDirection(id_mat)

    print("✓ Images flipped to standard orientation")

# Save oriented images
itk.imwrite(
    patient_image, str(output_dir / "patient_image_oriented.mha"), compression=True
)
itk.imwrite(
    patient_heart_mask,
    str(output_dir / "patient_heart_mask_oriented.mha"),
    compression=True,
)

Flipping image axes: True, True, False
✓ Images flipped to standard orientation


__array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
__array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
__array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword


## Convert Segmentation Mask to a Surface

In [6]:
contour_tools = ContourTools()
patient_heart_surface = contour_tools.extract_contours(patient_heart_mask)

## Perform Initial ICP Rigid Registration

Use ICP (Iterative Closest Point) with affine mode to align the model surface to the patient surface extracted from the segmentation mask. This provides a good initial alignment for the PCA-based registration.

The ICP registration pipeline:
1. Centroid alignment (automatic)
2. Rigid ICP alignment

The PCA registration will then refine this initial alignment with shape model constraints.

In [7]:
# Load the pca model
print("Loading template heart model...")
template_model = pv.read(str(heart_model_path))
template_model_surface = template_model.extract_surface()

icp_registrar = RegisterModelsICPITK(
    fixed_model=patient_heart_surface, reference_image=patient_image
)

icp_result = icp_registrar.register(
    moving_model=template_model_surface,
    transform_type="Affine",
    max_iterations=100,
    method="L-BFGS-B",
)

# Get the aligned mesh and transform
icp_registered_model_surface = icp_result["registered_model"]
icp_forward_point_transform = icp_result["forward_point_transform"]

print("\n✓ ICP affine registration complete")
print("   Transform =", icp_result["forward_point_transform"])

# Save aligned model
icp_registered_model_surface.save(str(output_dir / "icp_registered_model_surface.vtp"))
print("  Saved ICP-aligned model surface")

itk.transformwrite(
    [icp_result["forward_point_transform"]],
    str(output_dir / "icp_transform.hdf"),
    compression=True,
)
print("  Saved ICP transform")

Loading template heart model...


2026-01-25 18:09:31 INFO RegisterModelsICPITK ============================================================


2026-01-25 18:09:31 INFO RegisterModelsICPITK Affine Alignment Optimization


2026-01-25 18:09:31 INFO RegisterModelsICPITK ============================================================


2026-01-25 18:09:31 INFO RegisterModelsICPITK No initial transform provided, performing centroid alignment...


...

2026-01-25 18:13:37 INFO RegisterModelsICPITK Optimization completed!


2026-01-25 18:13:37 INFO RegisterModelsICPITK Final parameters: [-8.14300322e-02 -3.37112552e-02  5.60658146e-02  6.50784042e+02
  3.17801530e+02  9.56867714e+02  9.30311950e-01  8.66344398e-01
  8.00000000e-01 -3.00000000e-02 -2.21845401e-02  1.57768164e-02]


2026-01-25 18:13:37 INFO RegisterModelsICPITK Final mean distance: 2.14



✓ ICP affine registration complete
   Transform = ComposeScaleSkewVersor3DTransform (0000027D9E4A1EE0)
  RTTI typeinfo:   class itk::ComposeScaleSkewVersor3DTransform<double>
  Reference Count: 1
  Modified Time: 6414
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    0.922349 -0.119527 -0.0828587 
    0.108854 0.846143 0.137537 
    0.0538863 -0.145213 0.784111 
  Offset: [650.784, 317.802, 956.868]
  Center: [0, 0, 0]
  Translation: [650.784, 317.802, 956.868]
  Inverse: 
    1.05997 0.164018 0.0832395 
    -0.120883 1.12859 -0.210735 
    -0.0952307 0.197736 1.23058 
  Singular: 0
  Versor: [ -0.08143, -0.0337113, 0.0560658, 0.99453 ]
  Scale:       [0.930312, 0.866344, 0.8]
  Skew:        [-0.03, -0.0221845, 0.0157768]



  Saved ICP-aligned model surface
  Saved ICP transform


In [8]:
# Apply ICP transform to the full average mesh (not just surface)
# This gets the volumetric mesh into patient space for PCA registration
transform_tools = TransformTools()
icp_registered_model = transform_tools.transform_pvcontour(
    template_model, icp_forward_point_transform
)
icp_registered_model.save(str(output_dir / "icp_registered_model.vtk"))
print("\n✓ Applied ICP transform to full model mesh")


✓ Applied ICP transform to full model mesh


## Visualize Results


In [9]:
# Create side-by-side comparison
plotter = pv.Plotter(window_size=[600, 600])

plotter.add_mesh(patient_heart_surface, color="red", opacity=1.0, label="Patient")
plotter.add_mesh(
    icp_registered_model_surface, color="green", opacity=1.0, label="ICP Registered"
)
plotter.add_title("ICP Shape Fitting")
plotter.add_axes()

plotter.show()

Widget(value='<iframe src="http://localhost:49390/index.html?ui=P_0x27db368d960_0&reconnect=auto" class="pyvis…